You might need to install this on your system:

apt-get install python3-opencv git

In [ ]:
import os

if not os.path.isdir('k'):
  !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
  !cd k && git pull

!cd k && pip install .

Cloning into 'k'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 232 (delta 142), reused 122 (delta 58), pack-reused 0
Receiving objects: 100% (232/232), 186.09 KiB | 716.00 KiB/s, done.
Resolving deltas: 100% (142/142), done.
Processing /root/k
     |████████████████████████████████| 377 kB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 10.1 MB 27.0 MB/s eta 0:00:01
     |████████████████████████████████| 26.5 MB 44.4 MB/s eta 0:00:01
     |████████████████████████████████| 28.2 MB 33.9 MB/s eta 0:00:01
     |████████████████████████████████| 7.1 MB 46.8 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 268 kB 59.7 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 47.3 MB/s eta 0:00:01
     |████████████████████████████████| 13.1 MB 52.7 MB/s eta 0:00:01    |██████████████████

In [ ]:
!ls -l

total 108
-rw-r--r-- 1 root root 104382 Mar  4 18:16 YosukeBaseLineV4_various_mix.ipynb
drwxr-xr-x 6 root root    160 Mar  4 18:17 k
-rw-r--r-- 1 root root      0 Mar  4 18:10 onstart.log
-rw-r--r-- 1 root root     75 Mar  4 18:10 onstart.sh


In [ ]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

import skimage
print('skimage version',  skimage.__version__)
import tensorflow as tf
print(tf.__version__)
import sys
import cai
import cai.datasets
import cai.densenet
import cai.util
import cai.models

Python version
3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
Version info.
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
skimage version 0.16.2
2.2.0-dev20200303


Using TensorFlow backend.


In [ ]:
# Download from https://data.mendeley.com/datasets/tywbtsjrjv/1
url_zip_file="https://data.mendeley.com/datasets/tywbtsjrjv/1/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/Plant_leaf_diseases_dataset_without_augmentation.zip?dl=1"
local_zip_file="plant_leaf.zip"
expected_folder_name="plant_leaf"
Verbose=True
cai.datasets.download_zip_and_extract(
    url_zip_file=url_zip_file, local_zip_file=local_zip_file, 
    expected_folder_name=expected_folder_name, Verbose=Verbose)

Downloading:  https://data.mendeley.com/datasets/tywbtsjrjv/1/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/Plant_leaf_diseases_dataset_without_augmentation.zip?dl=1  to  plant_leaf.zip
Decompressing into:  plant_leaf


In [ ]:
import random
import os
import multiprocessing
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import np_utils
from keras.optimizers import SGD, adam
from keras.layers import *
from keras.models import Model
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!rm -r plant_leaf/Plant_leave_diseases_dataset_without_augmentation/Background_without_leaves -R
data_dir = "plant_leaf/Plant_leave_diseases_dataset_without_augmentation/"
print(os.listdir(data_dir))

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry___healthy', 'Cherry___Powdery_mildew', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___healthy', 'Corn___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___

In [ ]:
def load_plantvillage(seed=None, root_dir=None):
  if root_dir == None:
    print("define where the plantvillage folder is.")
    return
  random.seed(seed)
  def read_from_paths(paths):
    x=[]
    for path in paths:
      img = load_img(path, target_size=(224,224))
      img = img_to_array(img, dtype='float16')
      x.append(img)
    return x
  
  classes = os.listdir(root_dir)
  classes = sorted(classes)

  train_path = []
  val_path = []
  test_path = []

  train_x,train_y = [],[]
  val_x,val_y = [],[]
  test_x,test_y =[],[]

  #read path and categorize to three groups , 6,2,2
  for i,_class in enumerate(classes):
      paths = glob.glob(os.path.join(root_dir,_class,"*"))
      paths = [n for n in paths if n.endswith(".JPG") or n.endswith(".jpg")]
      random.shuffle(paths)
      cat_total = len(paths)
      train_path.extend(paths[:int(cat_total*0.6)])
      train_y.extend([i]*int(cat_total*0.6))
      val_path.extend(paths[int(cat_total*0.6):int(cat_total*0.8)])
      val_y.extend([i]*len(paths[int(cat_total*0.6):int(cat_total*0.8)]))
      test_path.extend(paths[int(cat_total*0.8):])
      test_y.extend([i]*len(paths[int(cat_total*0.8):]))

  print ("loading training images")
  train_x = read_from_paths(train_path)
  train_x = np.array(train_x, dtype='float16')/255.
  train_y = np.array(train_y)

  print ("loading validation images")
  val_x = read_from_paths(val_path)
  val_x = np.array(val_x, dtype='float16')/255.
  val_y = np.array(val_y)

  print ("loading testing images")
  test_x = read_from_paths(test_path)
  test_x = np.array(test_x, dtype='float16')/255.
  test_y = np.array(test_y)

  #calculate class weight
  classweight = class_weight.compute_class_weight('balanced', np.unique(train_y), train_y)

  #convert to categorical
  train_y = np_utils.to_categorical(train_y, 38)
  val_y = np_utils.to_categorical(val_y, 38)
  test_y = np_utils.to_categorical(test_y, 38)
  print("loaded")

  return train_x,val_x,test_x,train_y,val_y,test_y,classweight,classes

In [ ]:
def compiled_two_path_inception_v3(
  classes=1000, 
  input_shape=(224,224,3),
  two_paths_first_block=False,
  two_paths_second_block=False,
  max_mix_idx=10):
  base_model = cai.models.two_path_inception_v3(
    include_top=False,
    weights=None, #'two_paths_plant_leafs'
    input_shape=input_shape,
    pooling=None,
    two_paths_first_block=two_paths_first_block,
    two_paths_second_block=two_paths_second_block,
    max_mix_idx=max_mix_idx)
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(38, name='preprediction')(x)
  predictions = Activation('softmax',name='prediction')(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  model.compile(loss='categorical_crossentropy',
  optimizer="sgd",
  metrics=['accuracy','top_k_categorical_accuracy'])
  return model

In [ ]:
# test model compilation
model = compiled_two_path_inception_v3(classes=38)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = load_plantvillage(seed=7,root_dir=data_dir)

loading training images
loading validation images
loading testing images
loaded


In [ ]:
print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

(32570, 224, 224, 3) (10857, 224, 224, 3) (10876, 224, 224, 3)
(32570, 38) (10857, 38) (10876, 38)


In [ ]:
monitor='val_accuracy'
epochs=30
for max_mix_idx in range(-1,10,1):
    basefilename = 'Yosuke-Baseline-V4-'+str(max_mix_idx)
    print('Running: '+basefilename)
    model = compiled_two_path_inception_v3(
      classes=38,
      input_shape=(224,224,3),
      two_paths_first_block=False,
      two_paths_second_block=False,
      max_mix_idx=max_mix_idx)
    best_result_file_name = basefilename+'-best-result.hdf5'
    save_best = keras.callbacks.ModelCheckpoint(
      filepath=best_result_file_name, 
      monitor=monitor, 
      verbose=1, 
      save_best_only=True, 
      save_weights_only=False, 
      mode='max', 
      period=1)
    history = model.fit(train_x, train_y, epochs=epochs, batch_size=32,
      validation_data=(val_x,val_y),
      callbacks=[save_best],class_weight=classweight,
      workers=multiprocessing.cpu_count())
    print('Testing Last Model: '+basefilename)
    evaluated = model.evaluate(test_x,test_y)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Best Model Results: '+basefilename)
    model = keras.models.load_model(best_result_file_name, custom_objects={'CopyChannels': cai.layers.CopyChannels})
    evaluated = model.evaluate(test_x,test_y)
    #cai.models.save_model(model, basefilename)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
      print(name,metric)
    print('Finished: '+basefilename)

Running: Yosuke-Baseline-V4--1
Train on 32570 samples, validate on 10857 samples
Epoch 1/30
32570/32570 [==============================] - 139s 4ms/step - loss: 1.6607 - accuracy: 0.5693 - top_k_categorical_accuracy: 0.8420 - val_loss: 1.6382 - val_accuracy: 0.5548 - val_top_k_categorical_accuracy: 0.8473

Epoch 00001: val_accuracy improved from -inf to 0.55485, saving model to Yosuke-Baseline-V4--1-best-result.hdf5
Epoch 2/30
32570/32570 [==============================] - 136s 4ms/step - loss: 1.0792 - accuracy: 0.7224 - top_k_categorical_accuracy: 0.9463 - val_loss: 0.9314 - val_accuracy: 0.7449 - val_top_k_categorical_accuracy: 0.9614

Epoch 00002: val_accuracy improved from 0.55485 to 0.74487, saving model to Yosuke-Baseline-V4--1-best-result.hdf5
Epoch 3/30
32570/32570 [==============================] - 133s 4ms/step - loss: 0.8510 - accuracy: 0.7850 - top_k_categorical_accuracy: 0.9704 - val_loss: 0.8544 - val_accuracy: 0.7611 - val_top_k_categorical_accuracy: 0.9633

Epoch 00003

32570/32570 [==============================] - 136s 4ms/step - loss: 0.1554 - accuracy: 0.9645 - top_k_categorical_accuracy: 0.9989 - val_loss: 0.1467 - val_accuracy: 0.9630 - val_top_k_categorical_accuracy: 0.9993

Epoch 00027: val_accuracy improved from 0.95339 to 0.96297, saving model to Yosuke-Baseline-V4--1-best-result.hdf5
Epoch 28/30
32570/32570 [==============================] - 136s 4ms/step - loss: 0.1516 - accuracy: 0.9649 - top_k_categorical_accuracy: 0.9990 - val_loss: 0.2110 - val_accuracy: 0.9427 - val_top_k_categorical_accuracy: 0.9982

Epoch 00028: val_accuracy did not improve from 0.96297
Epoch 29/30
32570/32570 [==============================] - 134s 4ms/step - loss: 0.1489 - accuracy: 0.9659 - top_k_categorical_accuracy: 0.9991 - val_loss: 0.2004 - val_accuracy: 0.9447 - val_top_k_categorical_accuracy: 0.9982

Epoch 00029: val_accuracy did not improve from 0.96297
Epoch 30/30
32570/32570 [==============================] - 136s 4ms/step - loss: 0.1433 - accuracy: 0.9

32570/32570 [==============================] - 153s 5ms/step - loss: 0.1093 - accuracy: 0.9774 - top_k_categorical_accuracy: 0.9995 - val_loss: 0.2817 - val_accuracy: 0.9107 - val_top_k_categorical_accuracy: 0.9943

Epoch 00022: val_accuracy did not improve from 0.97228
Epoch 23/30
32570/32570 [==============================] - 153s 5ms/step - loss: 0.1044 - accuracy: 0.9782 - top_k_categorical_accuracy: 0.9995 - val_loss: 0.2506 - val_accuracy: 0.9232 - val_top_k_categorical_accuracy: 0.9969

Epoch 00023: val_accuracy did not improve from 0.97228
Epoch 24/30
32570/32570 [==============================] - 153s 5ms/step - loss: 0.1003 - accuracy: 0.9799 - top_k_categorical_accuracy: 0.9996 - val_loss: 0.0870 - val_accuracy: 0.9803 - val_top_k_categorical_accuracy: 0.9995

Epoch 00024: val_accuracy improved from 0.97228 to 0.98029, saving model to Yosuke-Baseline-V4-0-best-result.hdf5
Epoch 25/30
32570/32570 [==============================] - 152s 5ms/step - loss: 0.0954 - accuracy: 0.98

32570/32570 [==============================] - 170s 5ms/step - loss: 0.0926 - accuracy: 0.9804 - top_k_categorical_accuracy: 0.9995 - val_loss: 0.1263 - val_accuracy: 0.9639 - val_top_k_categorical_accuracy: 0.9986

Epoch 00017: val_accuracy did not improve from 0.96583
Epoch 18/30
32570/32570 [==============================] - 170s 5ms/step - loss: 0.0877 - accuracy: 0.9805 - top_k_categorical_accuracy: 0.9996 - val_loss: 0.1585 - val_accuracy: 0.9522 - val_top_k_categorical_accuracy: 0.9985

Epoch 00018: val_accuracy did not improve from 0.96583
Epoch 19/30
32570/32570 [==============================] - 170s 5ms/step - loss: 0.0805 - accuracy: 0.9819 - top_k_categorical_accuracy: 0.9997 - val_loss: 0.1211 - val_accuracy: 0.9649 - val_top_k_categorical_accuracy: 0.9994

Epoch 00019: val_accuracy did not improve from 0.96583
Epoch 20/30
32570/32570 [==============================] - 170s 5ms/step - loss: 0.0750 - accuracy: 0.9842 - top_k_categorical_accuracy: 0.9998 - val_loss: 0.0909 

32570/32570 [==============================] - 182s 6ms/step - loss: 0.1105 - accuracy: 0.9737 - top_k_categorical_accuracy: 0.9994 - val_loss: 0.4802 - val_accuracy: 0.8500 - val_top_k_categorical_accuracy: 0.9817

Epoch 00012: val_accuracy did not improve from 0.97117
Epoch 13/30
32570/32570 [==============================] - 183s 6ms/step - loss: 0.1025 - accuracy: 0.9768 - top_k_categorical_accuracy: 0.9996 - val_loss: 0.1215 - val_accuracy: 0.9632 - val_top_k_categorical_accuracy: 0.9990

Epoch 00013: val_accuracy did not improve from 0.97117
Epoch 14/30
32570/32570 [==============================] - 182s 6ms/step - loss: 0.0906 - accuracy: 0.9797 - top_k_categorical_accuracy: 0.9996 - val_loss: 0.1213 - val_accuracy: 0.9664 - val_top_k_categorical_accuracy: 0.9998

Epoch 00014: val_accuracy did not improve from 0.97117
Epoch 15/30
32570/32570 [==============================] - 183s 6ms/step - loss: 0.0852 - accuracy: 0.9806 - top_k_categorical_accuracy: 0.9998 - val_loss: 0.1449 

...


Epoch 00025: val_accuracy did not improve from 0.98996
Epoch 26/30
32570/32570 [==============================] - 230s 7ms/step - loss: 0.0118 - accuracy: 0.9975 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0733 - val_accuracy: 0.9767 - val_top_k_categorical_accuracy: 0.9998

Epoch 00026: val_accuracy did not improve from 0.98996
Epoch 27/30
32570/32570 [==============================] - 231s 7ms/step - loss: 0.0119 - accuracy: 0.9975 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0301 - val_accuracy: 0.9905 - val_top_k_categorical_accuracy: 0.9999

Epoch 00027: val_accuracy improved from 0.98996 to 0.99051, saving model to Yosuke-Baseline-V4-5-best-result.hdf5
Epoch 28/30
32570/32570 [==============================] - 231s 7ms/step - loss: 0.0093 - accuracy: 0.9982 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0278 - val_accuracy: 0.9912 - val_top_k_categorical_accuracy: 0.9999

Epoch 00028: val_accuracy improved from 0.99051 to 0.99116, saving model to Yosuke-Baseline

32570/32570 [==============================] - 252s 8ms/step - loss: 0.0156 - accuracy: 0.9967 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0660 - val_accuracy: 0.9802 - val_top_k_categorical_accuracy: 0.9997

Epoch 00021: val_accuracy did not improve from 0.98489
Epoch 22/30
32570/32570 [==============================] - 253s 8ms/step - loss: 0.0120 - accuracy: 0.9977 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0727 - val_accuracy: 0.9777 - val_top_k_categorical_accuracy: 0.9998

Epoch 00022: val_accuracy did not improve from 0.98489
Epoch 23/30
32570/32570 [==============================] - 255s 8ms/step - loss: 0.0131 - accuracy: 0.9970 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0408 - val_accuracy: 0.9875 - val_top_k_categorical_accuracy: 0.9997

Epoch 00023: val_accuracy improved from 0.98489 to 0.98747, saving model to Yosuke-Baseline-V4-6-best-result.hdf5
Epoch 24/30
32570/32570 [==============================] - 254s 8ms/step - loss: 0.0136 - accuracy: 0.99

32570/32570 [==============================] - 272s 8ms/step - loss: 0.0217 - accuracy: 0.9950 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0960 - val_accuracy: 0.9697 - val_top_k_categorical_accuracy: 0.9992

Epoch 00016: val_accuracy did not improve from 0.98084
Epoch 17/30
32570/32570 [==============================] - 273s 8ms/step - loss: 0.0184 - accuracy: 0.9959 - top_k_categorical_accuracy: 0.9999 - val_loss: 0.0545 - val_accuracy: 0.9819 - val_top_k_categorical_accuracy: 0.9999

Epoch 00017: val_accuracy improved from 0.98084 to 0.98195, saving model to Yosuke-Baseline-V4-7-best-result.hdf5
Epoch 18/30
32570/32570 [==============================] - 272s 8ms/step - loss: 0.0223 - accuracy: 0.9939 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0769 - val_accuracy: 0.9755 - val_top_k_categorical_accuracy: 0.9996

Epoch 00018: val_accuracy did not improve from 0.98195
Epoch 19/30
32570/32570 [==============================] - 274s 8ms/step - loss: 0.0182 - accuracy: 0.99

32570/32570 [==============================] - 288s 9ms/step - loss: 0.0354 - accuracy: 0.9895 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.3596 - val_accuracy: 0.9014 - val_top_k_categorical_accuracy: 0.9922

Epoch 00011: val_accuracy did not improve from 0.95717
Epoch 12/30
32570/32570 [==============================] - 285s 9ms/step - loss: 0.0248 - accuracy: 0.9934 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.1470 - val_accuracy: 0.9556 - val_top_k_categorical_accuracy: 0.9994

Epoch 00012: val_accuracy did not improve from 0.95717
Epoch 13/30
32570/32570 [==============================] - 284s 9ms/step - loss: 0.0226 - accuracy: 0.9942 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.1549 - val_accuracy: 0.9496 - val_top_k_categorical_accuracy: 0.9991

Epoch 00013: val_accuracy did not improve from 0.95717
Epoch 14/30
32570/32570 [==============================] - 287s 9ms/step - loss: 0.0204 - accuracy: 0.9947 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0724 

32570/32570 [==============================] - 300s 9ms/step - loss: 0.1184 - accuracy: 0.9634 - top_k_categorical_accuracy: 0.9992 - val_loss: 0.2515 - val_accuracy: 0.9196 - val_top_k_categorical_accuracy: 0.9968

Epoch 00006: val_accuracy improved from 0.89141 to 0.91959, saving model to Yosuke-Baseline-V4-9-best-result.hdf5
Epoch 7/30
32570/32570 [==============================] - 301s 9ms/step - loss: 0.0965 - accuracy: 0.9697 - top_k_categorical_accuracy: 0.9996 - val_loss: 0.3995 - val_accuracy: 0.8753 - val_top_k_categorical_accuracy: 0.9912

Epoch 00007: val_accuracy did not improve from 0.91959
Epoch 8/30
32570/32570 [==============================] - 303s 9ms/step - loss: 0.0751 - accuracy: 0.9770 - top_k_categorical_accuracy: 0.9997 - val_loss: 0.1283 - val_accuracy: 0.9570 - val_top_k_categorical_accuracy: 0.9987

Epoch 00008: val_accuracy improved from 0.91959 to 0.95699, saving model to Yosuke-Baseline-V4-9-best-result.hdf5
Epoch 9/30
32570/32570 [=======================